In [1]:
# Step 1: Install required packages
!pip install transformers torch pandas numpy scikit-learn requests

# Step 2: Import necessary libraries
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader
import requests
from IPython.display import clear_output

# Sample dataset (in practice, use real Codeforces problems)
# Format: Problem ID, Problem Statement, Tags
data = [
    (1, "Given an array... find maximum sum", ["greedy", "arrays"]),
    (2, "Tree node counting...", ["trees", "dp"]),
    (3, "Find shortest path...", ["graphs", "bfs"]),
    # Add more samples
]

# Convert to DataFrame
df = pd.DataFrame(data, columns=["id", "statement", "tags"])

# Step 3: Preprocess data
mlb = MultiLabelBinarizer()
tags_encoded = mlb.fit_transform(df["tags"])
num_tags = len(mlb.classes_)

# Step 4: Prepare dataset for BERT
class ProblemDataset(Dataset):
    def __init__(self, statements, tags, tokenizer, max_len):
        self.statements = statements
        self.tags = tags
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.statements)

    def __getitem__(self, idx):
        statement = str(self.statements[idx])
        encoding = self.tokenizer.encode_plus(
            statement,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.FloatTensor(self.tags[idx])
        }

# Step 5: Initialize BERT model
MODEL_NAME = 'bert-base-uncased'
MAX_LEN = 256
BATCH_SIZE = 8
EPOCHS = 3

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_tags)

# Split data
X_train, X_val, y_train, y_val = train_test_split(
    df["statement"], tags_encoded, test_size=0.2, random_state=42
)

train_dataset = ProblemDataset(X_train.values, y_train, tokenizer, MAX_LEN)
val_dataset = ProblemDataset(X_val.values, y_val, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# Step 6: Training setup
optimizer = AdamW(model.parameters(), lr=2e-5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Step 7: Training loop
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{EPOCHS} - Train loss: {avg_train_loss:.4f}")

# Step 8: Create algorithm knowledge base
algorithms_db = {
    "greedy": [
        {
            "name": "Kruskal's Algorithm",
            "time_complexity": "O(E log E)",
            "space_complexity": "O(E)",
            "description": "For minimum spanning trees"
        }
    ],
    "dp": [
        {
            "name": "Knapsack 0/1",
            "time_complexity": "O(nW)",
            "space_complexity": "O(W)",
            "description": "For subset selection problems"
        }
    ],
    # Add more algorithms
}

# Step 9: Recommendation function
def recommend_algorithm(problem_statement, time_constraint, space_constraint):
    # Predict tags
    inputs = tokenizer(
        problem_statement,
        padding=True,
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    probs = torch.sigmoid(outputs.logits)
    predicted_tags = (probs > 0.5).int().cpu().numpy()
    tags = mlb.inverse_transform(predicted_tags)[0]

    # Get matching algorithms
    candidates = []
    for tag in tags:
        if tag in algorithms_db:
            candidates.extend(algorithms_db[tag])

    # Filter by constraints
    valid_algorithms = []
    for algo in candidates:
        if satisfies_constraint(algo['time_complexity'], time_constraint) and \
           satisfies_constraint(algo['space_complexity'], space_constraint):
            valid_algorithms.append(algo)

    return valid_algorithms[:3]  # Return top 3 recommendations

# Helper function for constraint checking
def satisfies_constraint(algo_complexity, user_constraint):
    # Implement proper complexity comparison logic here
    return True  # Simplified version

# Step 10: User interface
def main():
    clear_output()
    print("Codeforces Algorithm Recommender")
    problem = input("Enter problem statement: ")
    time_constraint = input("Enter time constraint (e.g., O(n log n)): ")
    space_constraint = input("Enter space constraint (e.g., O(n)): ")

    recommendations = recommend_algorithm(problem, time_constraint, space_constraint)

    print("\nRecommended Algorithms:")
    for i, algo in enumerate(recommendations, 1):
        print(f"{i}. {algo['name']}")
        print(f"   Time: {algo['time_complexity']}")
        print(f"   Space: {algo['space_complexity']}")
        print(f"   Description: {algo['description']}\n")

# Run the system
if __name__ == "__main__":
    main()

Codeforces Algorithm Recommender
Enter problem statement: Given a string s, find the length of the longest substring without repeating characters
Enter time constraint (e.g., O(n log n)): O(n)
Enter space constraint (e.g., O(n)): O(k)

Recommended Algorithms:
1. Knapsack 0/1
   Time: O(nW)
   Space: O(W)
   Description: For subset selection problems

